### Landmark to feature : Top

In [1]:
from feature_process import *
import numpy as np
import os

In [2]:
def get_name(path):
    sp = path.split('/')
    name = sp[-2]
    if sp[-1].find('basal')!=-1:
        bt = 'basal'
    else:
        bt = 'treat'
    return name+'-'+bt+'.csv'

In [2]:
# get csv paths
paths  = []
for root,dirs,files in os.walk('G:/mice/dataset/landmark/top_crop_mars/20220401-WT_F/'):
    root = root.replace('\\','/')
    if(root.find('')!=-1): #select mice
        for f in files:
            if(f.find('.csv')!=-1): #select file 
                paths.append(root+'/'+f)

In [6]:
save_root = './geofeat/'
if not os.path.isdir(save_root):
    os.makedirs(save_root)
for path in paths:
    dlc1 = dlc(path, False)
    ## feature transfer
    feat = combine_feat(dlc1.raw)
    ## copy and rename
    # feat = dlc1.raw
    fname = get_name(path)
    np.savetxt(save_root+fname,feat,delimiter=",")

In [5]:
# select all features
sel_dist = None
# for i in range(6):
#     for j in range(i+1,7):
#         sel_dist.append([i,j])
sel_ang = []
for i in range(5):
    for j in range(i+1,6):
        for k in range(j+1,7):
            sel_ang.append([i,j,k])
            # sel_ang.append([i,k,j])
            # sel_ang.append([j,i,k])

In [6]:
# single process
save_root = './geofeat/'
path = './landmark/Cap-C973-basal.csv'
dlc1 = dlc(path, False)
feat = combine_feat(dlc1.raw, sel_dist, sel_ang)
fname = 'Cap-C973-basal.csv'
np.savetxt(save_root+fname,feat,delimiter=",")

### Landmark to feature : Front

In [ ]:
inpath = r'C:\Users\Administrator\Desktop\ACA-project\feat\featgen\after'
save_path = '../feat/featgen/exp1/'
postfix = '-basal.csv'
pythonenv = r'C:\Users\Administrator\miniconda3\envs\micelab\python.exe'

if not os.path.isdir(save_path):
    os.makedirs(save_path)

for root,dirs,files in os.walk(inpath):
    root = root.replace('\\','/')
    if(root.find('')!=-1): #select mice
        for f in files:
            if(f.find('.xml')!=-1): #select file 
                xmlpath = root+'/'+f
                csvpath = save_path + f.replace('.xml', postfix)
                os.system(pythonenv+' feature_process_front.py --inpath '+xmlpath+' --outpath '+csvpath)

### Cluster Output (Load Mat file)

In [5]:
import scipy.io
import numpy as np
import cv2
import imageio

In [6]:
seg = scipy.io.loadmat('./mat/seg.mat')['seg'].T
label = scipy.io.loadmat('./mat/label.mat')['label'].T

In [4]:
def seg_gif(vid_path, seg, label,out_path='./result'):
    '''
    save video clips gif to out_path by classes
    seg : segment points
    label : one-hot label
    '''
    cap = cv2.VideoCapture(vid_path)

    for i in range(len(label[0])):
        cpath = out_path+'/'+str(i)+'/'
        if not os.path.isdir(cpath):
            os.makedirs(cpath)

    vid_i=0
    imgs = []
    segi=1
    i=0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if not ret:
            break
        # frame in clip
        if i<seg[segi]:
            imgs.append(frame)
        # split point reached
        if i == seg[segi]:
            cpath = out_path+'/'+str(np.argmax(label[segi-1]))+'/'
            imageio.mimsave(cpath+"/"+str(vid_i)+".gif",imgs)
            imgs = []
            imgs.append(frame)
            vid_i += 1
            segi += 1
        i+=1
    cap.release()

In [5]:
seg_gif('./F2.avi',seg,label)